# 第 3 阶段：LangGraph 工作流 + 人机交互

本 Notebook 用于学习和调试第 3 阶段的核心功能：
- LangGraph 有状态工作流
- 学习工作流（规划、检索、出题、评分、反馈）
- 检查点（Checkpoint）机制
- 人机交互（Human-in-the-Loop）
- 工作流状态管理
- 流式输出工作流进度

**参考文档**: `docs/stage_03/`  
**参考脚本**: `scripts/test_workflow.py`

## 1. 环境准备

In [ ]:
# 添加项目根目录到 Python 路径
import sys
from pathlib import Path

backend_dir = Path.cwd()
if backend_dir.name == 'notebooks':
    backend_dir = backend_dir.parent

if str(backend_dir) not in sys.path:
    sys.path.insert(0, str(backend_dir))

print(f"✅ 项目根目录: {backend_dir}")

# 导入必要的模块
import uuid
from datetime import datetime
from config import get_logger

logger = get_logger(__name__)

## 2. 工作流基础概念

### 2.1 学习工作流节点

学习工作流包含以下节点：

```
开始 → 规划学习路径 → 知识检索 → 出题测试 → [等待用户答题] → 自动评分 → 反馈建议 → 结束
```

- **规划节点**: 分析学习主题，制定学习计划
- **检索节点**: 从知识库检索相关资料
- **出题节点**: 根据主题生成测试题
- **评分节点**: 自动评分用户答案
- **反馈节点**: 分析学习效果，提供改进建议

## 3. 启动学习工作流

In [ ]:
from workflows.study_flow_graph import start_study_flow

# 生成唯一的 thread_id
thread_id = f"notebook_test_{uuid.uuid4().hex[:8]}"
print(f"🆔 Thread ID: {thread_id}\n")

# 定义学习问题
user_question = "我想学习 Python 的基础知识，包括变量、数据类型和控制流"

print(f"📝 学习问题: {user_question}\n")
print("🔄 启动工作流...\n")

In [ ]:
# 启动工作流
result = start_study_flow(
    user_question=user_question,
    thread_id=thread_id
)

print("✅ 工作流启动成功！\n")
print(f"{'='*60}")
print("工作流状态")
print(f"{'='*60}")
print(f"当前步骤: {result.get('current_step')}")
print(f"更新时间: {result.get('updated_at')}")
print(f"{'='*60}")

## 4. 查看学习计划

In [ ]:
# 查看生成的学习计划
if result.get('learning_plan'):
    plan = result['learning_plan']
    
    print("\n📚 学习计划")
    print(f"{'='*60}")
    print(f"主题: {plan.get('topic')}")
    print(f"难度: {plan.get('difficulty')}")
    print(f"预计时间: {plan.get('estimated_time')} 分钟")
    
    print(f"\n学习目标 ({len(plan.get('objectives', []))})个):")
    for i, obj in enumerate(plan.get('objectives', []), 1):
        print(f"  {i}. {obj}")
    
    print(f"\n关键知识点 ({len(plan.get('key_points', []))})个):")
    for i, point in enumerate(plan.get('key_points', []), 1):
        print(f"  {i}. {point}")
    
    print(f"{'='*60}")
else:
    print("⚠️  未生成学习计划")

## 5. 查看练习题

In [ ]:
# 查看生成的练习题
if result.get('quiz'):
    quiz = result['quiz']
    questions = quiz.get('questions', [])
    
    print("\n📝 练习题")
    print(f"{'='*60}")
    print(f"题目数量: {len(questions)}")
    print(f"总分: {quiz.get('total_points')} 分")
    print(f"建议用时: {quiz.get('time_limit')} 分钟")
    print(f"{'='*60}\n")
    
    # 显示每道题
    for i, q in enumerate(questions, 1):
        print(f"\n第 {i} 题 ({q['points']} 分) - {q['type']}")
        print(f"{'─'*60}")
        print(f"问题: {q['question']}")
        
        if q.get('options'):
            print("\n选项:")
            for opt in q['options']:
                print(f"  {opt}")
        
        # 显示正确答案（仅用于调试）
        print(f"\n✅ 正确答案: {q.get('answer')}")
        print(f"💡 解析: {q.get('explanation', '无')}")
else:
    print("⚠️  未生成练习题")

## 6. 提交答案

### 6.1 准备答案

In [ ]:
# 准备用户答案
# 这里我们模拟用户答题，你可以修改答案来测试不同的评分结果

quiz = result.get('quiz', {})
questions = quiz.get('questions', [])

user_answers = {}

print("准备答案...\n")

for i, q in enumerate(questions):
    q_id = q['id']
    q_type = q['type']
    
    # 这里我们故意让第一题答对，其他题答错，来测试评分功能
    if i == 0:
        # 第一题答对
        user_answers[q_id] = q['answer']
        print(f"第 {i+1} 题: 答对 ✅")
    else:
        # 其他题答错
        if q_type == 'multiple_choice':
            wrong_options = ['A', 'B', 'C', 'D']
            if q['answer'] in wrong_options:
                wrong_options.remove(q['answer'])
            user_answers[q_id] = wrong_options[0]
            print(f"第 {i+1} 题: 答错 ❌ (选择了 {wrong_options[0]})")
        else:
            user_answers[q_id] = "错误答案"
            print(f"第 {i+1} 题: 答错 ❌")

print(f"\n共准备 {len(user_answers)} 个答案")

### 6.2 提交答案并评分

In [ ]:
from workflows.study_flow_graph import submit_answers

print("✍️  提交答案...\n")

# 提交答案
result = submit_answers(
    thread_id=thread_id,
    user_answers=user_answers
)

print("✅ 答案提交成功！\n")

## 7. 查看评分结果

In [ ]:
# 查看评分结果
if result.get('score') is not None:
    score = result['score']
    score_details = result.get('score_details', {})
    
    print("\n📊 评分结果")
    print(f"{'='*60}")
    print(f"总分: {score} 分")
    print(f"答对: {score_details.get('correct_count', 0)}/{score_details.get('total_count', 0)} 题")
    print(f"正确率: {score_details.get('correct_count', 0) / score_details.get('total_count', 1) * 100:.1f}%")
    print(f"是否需要重试: {'是' if result.get('should_retry') else '否'}")
    print(f"{'='*60}")
    
    # 显示每题的评分详情
    if score_details.get('question_scores'):
        print("\n详细评分:")
        for q_score in score_details['question_scores']:
            status = "✅" if q_score.get('correct') else "❌"
            print(f"  {status} 题目 {q_score.get('question_id')}: {q_score.get('score', 0)} 分")
            if q_score.get('comment'):
                print(f"     评语: {q_score['comment']}")
else:
    print("⚠️  未获取到评分结果")

## 8. 查看反馈建议

In [ ]:
# 查看反馈
if result.get('feedback'):
    print("\n💬 学习反馈")
    print(f"{'='*60}")
    print(result['feedback'])
    print(f"{'='*60}")
else:
    print("⚠️  未获取到反馈")

## 9. 工作流状态管理

### 9.1 查询当前状态

In [ ]:
from workflows.study_flow_graph import get_workflow_state

# 查询工作流状态
state = get_workflow_state(thread_id)

if state:
    print("\n📊 工作流状态")
    print(f"{'='*60}")
    print(f"Thread ID: {thread_id}")
    print(f"当前步骤: {state.get('current_step')}")
    print(f"更新时间: {state.get('updated_at')}")
    print(f"是否完成: {'是' if state.get('is_finished') else '否'}")
    
    if state.get('error'):
        print(f"错误: {state['error']}")
    
    print(f"{'='*60}")
    
    # 显示状态中的关键数据
    print("\n状态数据:")
    print(f"  - 有学习计划: {'是' if state.get('learning_plan') else '否'}")
    print(f"  - 有练习题: {'是' if state.get('quiz') else '否'}")
    print(f"  - 有用户答案: {'是' if state.get('user_answers') else '否'}")
    print(f"  - 有评分: {'是' if state.get('score') is not None else '否'}")
    print(f"  - 有反馈: {'是' if state.get('feedback') else '否'}")
else:
    print("⚠️  无法获取工作流状态")

### 9.2 查看工作流历史

In [ ]:
from workflows.study_flow_graph import get_workflow_history

# 查看工作流历史
history = get_workflow_history(thread_id)

print(f"\n📜 工作流历史 (共 {len(history)} 个检查点)")
print(f"{'='*60}\n")

for i, h in enumerate(history, 1):
    print(f"{i}. {h.get('step', 'unknown')}")
    print(f"   时间: {h.get('timestamp', 'unknown')}")
    print(f"   配置: {h.get('config', {})}")
    print()

## 10. 检查点恢复测试

### 10.1 创建新的工作流并暂停

In [ ]:
# 创建新的工作流用于测试检查点恢复
recovery_thread_id = f"recovery_test_{uuid.uuid4().hex[:8]}"

print(f"🆔 新 Thread ID: {recovery_thread_id}\n")
print("启动工作流（将在答题环节暂停）...\n")

recovery_result = start_study_flow(
    user_question="学习机器学习的基础概念",
    thread_id=recovery_thread_id
)

print("✅ 工作流已暂停在答题环节")
print(f"当前步骤: {recovery_result.get('current_step')}")

### 10.2 模拟程序重启，从检查点恢复

In [ ]:
print("\n🔄 模拟程序重启，从检查点恢复状态...\n")

# 从检查点恢复状态
recovered_state = get_workflow_state(recovery_thread_id)

if recovered_state:
    print("✅ 成功从检查点恢复状态！\n")
    
    print("恢复的状态:")
    print(f"  步骤: {recovered_state.get('current_step')}")
    print(f"  更新时间: {recovered_state.get('updated_at')}")
    
    # 验证数据完整性
    data_integrity = []
    
    if recovered_state.get('quiz'):
        data_integrity.append("✅ 练习题数据完整")
    else:
        data_integrity.append("❌ 练习题数据丢失")
    
    if recovered_state.get('learning_plan'):
        data_integrity.append("✅ 学习计划数据完整")
    else:
        data_integrity.append("❌ 学习计划数据丢失")
    
    print("\n数据完整性检查:")
    for check in data_integrity:
        print(f"  {check}")
    
    print("\n✅ 检查点恢复测试完成！")
else:
    print("❌ 无法恢复状态")

## 11. 重试机制测试

In [ ]:
# 测试重试机制（得分低于60分时触发）
retry_thread_id = f"retry_test_{uuid.uuid4().hex[:8]}"

print(f"🆔 Thread ID: {retry_thread_id}\n")
print("启动工作流...\n")

retry_result = start_study_flow(
    user_question="学习深度学习的基本概念",
    thread_id=retry_thread_id
)

print("✅ 工作流启动成功\n")

# 故意全部答错
print("✍️  提交全错答案（测试重试机制）...\n")

quiz = retry_result.get('quiz', {})
questions = quiz.get('questions', [])

# 全部答错
wrong_answers = {}
for q in questions:
    q_id = q['id']
    q_type = q['type']
    
    if q_type == 'multiple_choice':
        wrong_options = ['A', 'B', 'C', 'D']
        if q['answer'] in wrong_options:
            wrong_options.remove(q['answer'])
        wrong_answers[q_id] = wrong_options[0]
    else:
        wrong_answers[q_id] = "错误答案"

retry_result = submit_answers(retry_thread_id, wrong_answers)

score = retry_result.get('score', 0)
should_retry = retry_result.get('should_retry', False)

print(f"\n📊 评分结果: {score} 分")
print(f"🔄 是否触发重试: {'是' if should_retry else '否'}")

if should_retry:
    print("\n✅ 重试机制正常工作！")
    print("   系统已自动生成新的练习题")
    
    # 查看新题目
    state = get_workflow_state(retry_thread_id)
    if state.get('quiz'):
        new_quiz = state['quiz']
        print(f"   新练习题: {len(new_quiz.get('questions', []))} 题")
else:
    print("\n⚠️  未触发重试（可能得分高于60分）")

## 12. 流式输出工作流进度

In [ ]:
# 演示流式输出工作流进度
from workflows.study_flow_graph import create_study_workflow

stream_thread_id = f"stream_test_{uuid.uuid4().hex[:8]}"

print(f"🆔 Thread ID: {stream_thread_id}\n")
print("📡 流式输出工作流进度:\n")
print(f"{'='*60}\n")

# 创建工作流
workflow = create_study_workflow()

# 配置
config = {"configurable": {"thread_id": stream_thread_id}}

# 输入
inputs = {
    "user_question": "学习 Python 函数的基础知识",
    "user_id": "test_user"
}

# 流式执行
for event in workflow.stream(inputs, config):
    print(f"事件: {event}")
    print()

print(f"{'='*60}")
print("\n✅ 工作流执行完成（已暂停在答题环节）")

## 13. 完整的工作流测试函数

In [ ]:
def test_complete_workflow(topic: str):
    """测试完整的学习工作流"""
    
    thread_id = f"complete_test_{uuid.uuid4().hex[:8]}"
    
    print(f"\n{'='*60}")
    print(f"完整工作流测试: {topic}")
    print(f"Thread ID: {thread_id}")
    print(f"{'='*60}\n")
    
    # 1. 启动工作流
    print("1️⃣  启动工作流...")
    result = start_study_flow(user_question=topic, thread_id=thread_id)
    print("   ✅ 工作流启动成功\n")
    
    # 2. 显示学习计划
    if result.get('learning_plan'):
        plan = result['learning_plan']
        print("2️⃣  学习计划:")
        print(f"   主题: {plan.get('topic')}")
        print(f"   难度: {plan.get('difficulty')}")
        print(f"   目标数: {len(plan.get('objectives', []))}\n")
    
    # 3. 显示练习题
    if result.get('quiz'):
        quiz = result['quiz']
        print("3️⃣  练习题:")
        print(f"   题目数: {len(quiz.get('questions', []))}")
        print(f"   总分: {quiz.get('total_points')}\n")
    
    # 4. 模拟答题（全部答对）
    print("4️⃣  提交答案（全部答对）...")
    questions = quiz.get('questions', [])
    correct_answers = {q['id']: q['answer'] for q in questions}
    
    result = submit_answers(thread_id, correct_answers)
    print("   ✅ 答案提交成功\n")
    
    # 5. 显示评分
    print("5️⃣  评分结果:")
    print(f"   得分: {result.get('score', 0)} 分")
    print(f"   是否需要重试: {'是' if result.get('should_retry') else '否'}\n")
    
    # 6. 显示反馈
    if result.get('feedback'):
        print("6️⃣  反馈:")
        print(f"   {result['feedback'][:200]}...\n")
    
    print(f"{'='*60}")
    print("✅ 完整工作流测试完成！")
    print(f"{'='*60}\n")
    
    return thread_id

# 使用示例
# test_thread = test_complete_workflow("学习 JavaScript 的异步编程")

## 14. 总结

通过本 Notebook，你已经学习了：

✅ LangGraph 工作流的基本概念  
✅ 学习工作流的完整流程（规划→检索→出题→评分→反馈）  
✅ 工作流的启动与状态管理  
✅ 检查点（Checkpoint）机制与状态恢复  
✅ 人机交互（Human-in-the-Loop）  
✅ 重试机制（得分低于60分）  
✅ 流式输出工作流进度  
✅ 工作流历史记录查询

**下一步**: 学习第 4 阶段 - DeepAgents 深度研究（`stage_04_deep_research.ipynb`）